### TRAIN

In [16]:
shots = [5]
xml_path = "/home/BI/luunvt/image_retrieval/mmfewshot/data/VOCdevkit/VOC2012/Annotations"
image_xml_path = "/home/BI/luunvt/image_retrieval/mmfewshot/data/VOCdevkit/VOC2012/JPEGImages"
few_shot_ann_path = "/home/BI/luunvt/image_retrieval/mmfewshot/data/few_shot_ann/voc"

In [17]:
from PIL import Image

def get_xml_content(cat, img_path, x1, y1, x2, y2):
  img_name = img_path.split("/")[-1]
  img = Image.open(img_path)
  w, h = img.size
  content = f'''
<annotation>
<filename>{img_name}</filename>
<folder>VOC2012</folder>
<object>
  <name>{cat}</name>
  <actions>
    <jumping>0</jumping>
    <other>1</other>
    <phoning>0</phoning>
    <playinginstrument>0</playinginstrument>
    <reading>0</reading>
    <ridingbike>0</ridingbike>
    <ridinghorse>0</ridinghorse>
    <running>0</running>
    <takingphoto>0</takingphoto>
    <usingcomputer>0</usingcomputer>
    <walking>0</walking>
  </actions>
  <bndbox>
    <xmax>{x2}</xmax>
    <xmin>{x1}</xmin>
    <ymax>{y2}</ymax>
    <ymin>{y1}</ymin>
  </bndbox>
  <difficult>0</difficult>
  <pose>Unspecified</pose>
  <point>
    <x>308</x>
    <y>189</y>
  </point>
</object>
<segmented>0</segmented>
<size>
  <depth>3</depth>
  <height>{h}</height>
  <width>{w}</width>
</size>
<source>
  <annotation>PASCAL VOC2012</annotation>
  <database>The VOC2012 Database</database>
  <image>flickr</image>
</source>
</annotation>
  '''.strip()
  return content

In [18]:
import mmdet
import os
import shutil
import cv2
import numpy as np

def get_bb(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    ## (2) Morph-op to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    ## (3) Find the max-area contour
    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    cnt = sorted(cnts, key=cv2.contourArea)[-1]

    ## (4) Crop and save it
    x,y,w,h = cv2.boundingRect(cnt)
    return x, y, x+w, y+h

def create_annotations_and_label_images(cat, query_image_path,
                                        label_image_path,
                                        ):
    color = (0, 0, 255)
    thickness = 2
    if not os.path.exists(os.path.join(label_image_path, "img_with_bbox")):
        os.makedirs(os.path.join(label_image_path, "img_with_bbox"))

    for idx, img_name in enumerate(os.listdir(query_image_path)):
        img_path = os.path.join(query_image_path, img_name)
        x1, y1, x2, y2 = get_bb(img_path)

        img = cv2.imread(img_path)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        new_name = img_name.split(".")[0] + "_bbox." + img_name.split(".")[1]
        new_img_path = os.path.join(label_image_path, "img_with_bbox", new_name)
        cv2.imwrite(new_img_path, img)
        xml_content = get_xml_content(cat, img_path, x1, y1, x2, y2)
        xml_name = img_name.split(".")[0]
        if os.path.exists(os.path.join(xml_path, xml_name + ".xml")):
            os.remove(os.path.join(xml_path, xml_name + ".xml"))
        f = open(os.path.join(xml_path, xml_name + ".txt"), 'w')
        f.write(xml_content)
        f.close()
        os.rename(os.path.join(xml_path, xml_name + ".txt"), os.path.join(xml_path, xml_name + ".xml"))
        
        shutil.copy(os.path.join(query_image_path, img_name), os.path.join(image_xml_path, img_name))

In [19]:
def set_category(cat):
    with open('/home/BI/luunvt/image_retrieval/mmfewshot/mmfewshot/detection/datasets/voc.py', 'r') as file:
        file_content = file.readlines()

    file_content[23] = f"\t\t\t\t\t'pottedplant', 'sheep', 'train', 'tvmonitor', '{cat}'),\n"
    file_content[33] = f"\tNOVEL_CLASSES_SPLIT1=('{cat}',),\n"

    with open('/home/BI/luunvt/image_retrieval/mmfewshot/mmfewshot/detection/datasets/voc.py', 'w') as file:
        file.writelines(file_content)
    print(file_content[33])

In [20]:
def add_image_to_txt_path(cat, query_image_path):
  for shot in shots:
      if (len(os.listdir(query_image_path))) < shot:
          print(f"[INFO] not enough image for {shot} shot")
          break
      print(shot)
      f = open(os.path.join(few_shot_ann_path, f"benchmark_{str(shot)}shot/box_{str(shot)}shot_{cat}_train.txt"), "w")
      for img_name in os.listdir(query_image_path)[:shot]:
        f.write(f"VOC2012/JPEGImages/{img_name}\n")
      f.close()

In [1]:
import subprocess

data_dict = [
    {
        'cat': '3326',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/3326/focus_3326",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/3326/3326",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/3326/",
    },
    {
        'cat': '3191',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/3191/focus_3191",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/3191/3191",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/3191/",
    },
    {
        'cat': 'door',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/img/focus_door",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/img/door",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/img/",
    },
    {
        'cat': 'kettlebell',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/image/KETTLEBELL/focus_Kettlebell",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/image/KETTLEBELL/Kettlebell",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/image/KETTLEBELL/",
    },
    {
        'cat': '3190',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/3190/focus_3190",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/3190/3190",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/3190/",
    },
    {
        'cat': '2930',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/2930/focus_2930",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/2930/2930",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/2930/",
    },
    {
        'cat': '2864',
        'query_image_path': "/home/BI/luunvt/image_retrieval/data/2864/focus_2864",
        'database_image_path': "/home/BI/luunvt/image_retrieval/data/2864/2864",
        'label_image_path': "/home/BI/luunvt/image_retrieval/data/2864/",
    }
]

work_dir = "work_dirs/pipeline_v2/fine_tune_4"

In [23]:
for data in data_dict:
  print("Processing data ", data['cat'], "...")
  create_annotations_and_label_images(data['cat'], data['query_image_path'],
                                      data['label_image_path'])
  set_category(data['cat'])
  add_image_to_txt_path(data['cat'], data['query_image_path'])
  command = f"CUDA_VISIBLE_DEVICES=0 python tools/detection/train.py configs/detection/fsce/voc/split1/fsce_r101_fpn_contrastive-loss_voc-split1_{str(shots[0])}shot-fine-tuning.py --work-dir {os.path.join(work_dir, data['cat'])} --no-validate"
  result = subprocess.run(command, shell=True) 

Processing data  3326 ...
	NOVEL_CLASSES_SPLIT1=('3326',),

5


2023-04-17 13:35:48,355 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_

Processing data  3191 ...
	NOVEL_CLASSES_SPLIT1=('3191',),

5


2023-04-17 13:47:19,479 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_

Processing data  door ...
	NOVEL_CLASSES_SPLIT1=('door',),

5


...

2023-04-17 14:22:15,283 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_

Processing data  2930 ...
	NOVEL_CLASSES_SPLIT1=('2930',),

5


2023-04-17 14:33:42,142 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_

Processing data  2864 ...
	NOVEL_CLASSES_SPLIT1=('2864',),

5


2023-04-17 14:45:17,343 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_

### INFERENCE

In [4]:
folder_workdir = "/home/BI/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v2/fine_tune_0"
print(os.listdir(folder_workdir))

['3326_newbbox', 'door', '2930_newbbox', '3191_newbbox', 'door_newbbox', 'note.txt', 'kettlebell', '3190_newbbox', '2864_newbbox', '3190']


In [24]:
from mmfewshot.detection.apis import init_detector, inference_detector
import mmcv
import os
from tqdm import tqdm
import time
import cv2
import torch
import torchvision
import numpy as np

THR = 0.5
THR_NMS = 0.2

folder_workdir = "/home/BI/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v2/fine_tune_4"
save_path = "/home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_4"
for data in data_dict:
  # Specify the path to model config and checkpoint file
  config_file = os.path.join(folder_workdir, data['cat'], "fsce_r101_fpn_contrastive-loss_voc-split1_5shot-fine-tuning.py")
  checkpoint_file = os.path.join(folder_workdir, data['cat'], "iter_4500.pth")

  # build the model from a config file and a checkpoint file
  model = init_detector(config_file, checkpoint_file, device='cuda:0')

  img_path = data['database_image_path']
  cat = data['cat']
  # breakpoint()
  num_iter = int(checkpoint_file.split("/")[-1].split(".")[0].split("_")[1])
  folder_name = f"{cat}_{num_iter}iter_thr{str(int(THR*100))}_nms{str(int(THR_NMS*100))}"
  print(folder_name)
  include_path = os.path.join(save_path, folder_name, "include")
  if not os.path.exists(include_path):
    print(f"Make dir {include_path}")
    os.makedirs(include_path)

  exclude_path = os.path.join(save_path, folder_name, "exclude")
  if not os.path.exists(exclude_path):
    print(f"Make dir {exclude_path}")
    os.makedirs(exclude_path)

  sum_time = 0

  def nms(boxes):
    boxbox = torch.from_numpy(boxes[:, 0:4])
    score = torch.from_numpy(boxes[:, 4])
    result = torchvision.ops.nms(boxbox, score, THR_NMS)
    return result.numpy()

  for im_name in tqdm(os.listdir(img_path)):
    img = os.path.join(img_path, im_name)
    st = time.time()
    try:
      result = inference_detector(model, img)
    except:
      print(im_name)
      continue
    sum_time += (time.time() - st)
    img = cv2.imread(img)
    boxes = np.array([box for box in result[-1] if box[4] >= THR])
    if len(boxes) > 0:
      box_idxs = nms(boxes)
      for box in boxes[box_idxs]:
        score = box[4]
        if score >= THR:
          box = box.astype(int)
          color = (0, 0, 255)
          thickness = 2
          img = cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, thickness)
          img = cv2.putText(img, str(score), (box[0], box[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
      cv2.imwrite(os.path.join(include_path, im_name), img)
    else:
      cv2.imwrite(os.path.join(exclude_path, im_name), img)
  print(f"Infer on {len(os.listdir(img_path))} got {sum_time}s")


load checkpoint from local path: /home/BI/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v2/fine_tune_4/3326/iter_4500.pth
3326_4500iter_thr50_nms20
Make dir /home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_4/3326_4500iter_thr50_nms20/include
Make dir /home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_4/3326_4500iter_thr50_nms20/exclude


  0%|          | 0/10730 [00:00<?, ?it/s]/home/BI/anaconda3/envs/mmfewshot/lib/python3.8/site-packages/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
 61%|██████    | 6546/10730 [06:41<03:29, 19.93it/s]

B078YZLBNF.jpg


100%|██████████| 10730/10730 [10:51<00:00, 16.47it/s]


Infer on 10730 got 618.864869594574s
load checkpoint from local path: /home/BI/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v2/fine_tune_4/3191/iter_4500.pth
3191_4500iter_thr50_nms20
Make dir /home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_4/3191_4500iter_thr50_nms20/include
Make dir /home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_4/3191_4500iter_thr50_nms20/exclude


...

 98%|█████████▊| 10275/10452 [11:42<00:10, 16.77it/s]

B0B198J9YH.jpg


100%|██████████| 10452/10452 [11:55<00:00, 14.62it/s]

Infer on 10452 got 645.1860456466675s


: 

### MISCELLANEOUS

In [ ]:
#!/usr/bin/python3
# Created by Silencer @ Stackoverflow 
# 2018.01.23 14:41:42 CST
# 2018.01.23 18:17:42 CST
import cv2
import numpy as np

## (1) Convert to gray, and threshold
img = cv2.imread("/home/BI/luunvt/image_retrieval/data/photo_2023-04-11_08-17-05.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

## (2) Morph-op to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

## (3) Find the max-area contour
cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

## (4) Crop and save it
x,y,w,h = cv2.boundingRect(cnt)
# dst = img[y:y+h, x:x+w]
color = (255, 0, 0)
thickness = 1
img = cv2.rectangle(img, (y, x), (y+h, x+w), color, thickness)
cv2.imshow("img.jpg", img)
# cv2.imwrite("001.png", dst)

In [ ]:
import mmdet

In [13]:
import os
import shutil

folder1 = "/home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_v0/door_new_bbox_4500iter_thr20/include"
folder2 = "/home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_v0/door_new_bbox_4500iter_th10_nms40/include"

list_1 = os.listdir(folder1)
list_2 = os.listdir(folder2)

diff_folder = "/home/BI/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/diff_20_10"

list_diff = list(set(list_2) - set(list_1))
for im in list_diff:
    source = os.path.join(folder2, im)
    dest = os.path.join(diff_folder, im)
    shutil.copy(source, dest)

In [ ]:
a, b = map(int, input().split())
def recur(a, b):
    if b == 0:
        return a
    return recur(b, b % a)
c = recur(a, b)
print(c)